#### Import block

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant

#### Function definitions

In [2]:
# Loader functions
# Inspiration: https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a

def get_input(path):
    # Load array.
    t_input = np.load(path)
    # Pad to even number of pixels
    t_input = np.pad(t_input, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    t_input = tf.expand_dims(t_input, axis = -1)
    return t_input

def get_output(path):
    # Load array.
    t_output = np.load(path)
    # Pad to even number of pixels
    t_output = np.pad(t_output, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    t_output = tf.expand_dims(t_output, axis = -1)
    return t_output

def data_generator(samples, batch_size = 64):
    
    while True:
          # Select files (paths/indices) for the batch
          batch_samples  = np.random.choice(a = samples.index, 
                                          size = batch_size)
          batch_input  = []
          batch_output = [] 
          
          # Read in each input, perform preprocessing and get labels
          for sample in batch_samples:
              input = get_input(samples.loc[sample].features)
              output = get_output(samples.loc[sample].labels)
            
              batch_input += [input]
              batch_output += [output]
          # Return a tuple of (input, output) to feed the network
          batch_x = np.array(batch_input)
          batch_y = np.array(batch_output)
        
          yield(batch_x, batch_y)


In [3]:
m = pd.read_csv('Sample_Dataset/meta.csv')
m

,Unnamed: 0,Lon,Lat,Date,features,labels
0,356962,-53.345535,-6.535028,2017-07-30,Sample_Dataset/train_features/356962.npy,Sample_Dataset/train_labels/356962.npy
1,546517,-47.329685,-8.260676,2017-08-20,Sample_Dataset/train_features/546517.npy,Sample_Dataset/train_labels/546517.npy
2,799359,-50.967861,-8.255809,2017-09-04,Sample_Dataset/train_features/799359.npy,Sample_Dataset/train_labels/799359.npy
3,714590,-66.144379,-12.624272,2017-08-31,Sample_Dataset/train_features/714590.npy,Sample_Dataset/train_labels/714590.npy
4,240012,-48.350338,-11.838207,2017-07-14,Sample_Dataset/train_features/240012.npy,Sample_Dataset/train_labels/240012.npy
...,...,...,...,...,...,...
645,1983526,-51.431313,0.659630,2017-11-26,Sample_Dataset/train_features/1983526.npy,Sample_Dataset/train_labels/1983526.npy
646,2079183,-45.239655,-4.660801,2017-12-12,Sample_Dataset/train_features/2079183.npy,Sample_Dataset/train_labels/2079183.npy
647,1934639,-47.025230,-2.296389,2017-11-18,Sample_Dataset/train_features/1934639.npy,Sample_Dataset/train_labels/1934639.npy
648,1417977,-53.128605,-2.962739,2017-09-30,Sample_Dataset/train_features/1417977.npy,Sample_Dataset/train_labels/1417977.npy


#### Load data

In [4]:
# Get data from one chip
sample_input = np.load('Sample_Dataset/train_features/3243.npy')

# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
a = tf.expand_dims(a, axis = -1)
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)
# Display shape for verification.
a.shape

TensorShape([1, 16, 256, 256, 1])

#### Experiment with basic model structure

In [5]:
b = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01), 
                           input_shape=(a))(a)
b.shape

TensorShape([1, 16, 256, 256, 32])

In [6]:
c = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01) 
                           )(b)
c.shape

TensorShape([1, 16, 256, 256, 32])

In [7]:
d = layers.MaxPooling3D((2,4,4))(c)
d.shape

TensorShape([1, 8, 64, 64, 32])

In [8]:
e = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(d)
e.shape

TensorShape([1, 8, 64, 64, 64])

In [9]:
f = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(e)
f.shape

TensorShape([1, 8, 64, 64, 64])

In [10]:
g = layers.MaxPooling3D((2,2,2))(f)
g.shape

TensorShape([1, 4, 32, 32, 64])

In [11]:
h = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(g)
h.shape

TensorShape([1, 4, 32, 32, 128])

In [12]:
i = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(h)
i.shape

TensorShape([1, 4, 32, 32, 128])

In [13]:
j = layers.MaxPooling3D((2,2,2))(i)
j.shape

TensorShape([1, 2, 16, 16, 128])

In [14]:
k = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(j)
k.shape

TensorShape([1, 2, 16, 16, 256])

In [15]:
l = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(k)
l.shape

TensorShape([1, 2, 16, 16, 256])

In [16]:
m = layers.MaxPooling3D((1,4,4))(l)
m.shape

TensorShape([1, 2, 4, 4, 256])

In [17]:
n = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(m)
n.shape

TensorShape([1, 2, 4, 4, 512])

In [18]:
o = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(n)
o.shape

TensorShape([1, 2, 4, 4, 512])

In [19]:
p = layers.MaxPooling3D((1,2,2))(o)
p.shape

TensorShape([1, 2, 2, 2, 512])

In [20]:
q = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu')(p)
q.shape

TensorShape([1, 2, 2, 2, 1024])

In [21]:
r = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu')(q)
r.shape

TensorShape([1, 2, 2, 2, 1024])

In [22]:
s = layers.MaxPooling3D((2,2,2))(r)
s.shape

TensorShape([1, 1, 1, 1, 1024])

In [23]:
t = layers.Conv3DTranspose(512, (2,2,2), (2,2,2), padding = 'same', activation='relu')(s)
t = layers.concatenate([t, p])
t.shape

TensorShape([1, 2, 2, 2, 1024])

In [24]:
u = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(t)
u.shape

TensorShape([1, 2, 2, 2, 512])

In [25]:
v = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(u)
v.shape

TensorShape([1, 2, 2, 2, 512])

In [26]:
w = layers.Conv3DTranspose(512, (2,2,2), (1,2,2), padding = 'same', activation='relu')(v)
w = layers.concatenate([w, o])
w.shape

TensorShape([1, 2, 4, 4, 1024])

In [27]:
x = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(w)
x.shape

TensorShape([1, 2, 4, 4, 256])

In [28]:
y = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(x)
y.shape

TensorShape([1, 2, 4, 4, 256])

In [29]:
z = layers.Conv3DTranspose(256, (2,2,2), (1,4,4), padding = 'same', activation='relu')(y)
z = layers.concatenate([z, l])
z.shape

TensorShape([1, 2, 16, 16, 512])

In [30]:
aa = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(z)
aa.shape

TensorShape([1, 2, 16, 16, 256])

In [31]:
ab = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(aa)
ab.shape

TensorShape([1, 2, 16, 16, 256])

In [32]:
ac = layers.Conv3DTranspose(128, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ab)
ac = layers.concatenate([ac, i])
ac.shape

TensorShape([1, 4, 32, 32, 256])

In [33]:
ad = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(ac)
ad.shape

TensorShape([1, 4, 32, 32, 128])

In [34]:
ae = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(ad)
ae.shape

TensorShape([1, 4, 32, 32, 128])

In [35]:
af = layers.Conv3DTranspose(64, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ae)
af = layers.concatenate([af, f])
af.shape

TensorShape([1, 8, 64, 64, 128])

In [36]:
ag = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(af)
ag.shape

TensorShape([1, 8, 64, 64, 64])

In [37]:
ah = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(ag)
ah.shape

TensorShape([1, 8, 64, 64, 64])

In [38]:
ai = layers.Conv3DTranspose(32, (2,2,2), (1,4,4), padding = 'same', activation='relu')(ah)
ag = layers.MaxPooling3D((2,1,1))(c)
ai = layers.concatenate([ai, ag])
ai.shape

TensorShape([1, 8, 256, 256, 64])

In [39]:
ak = layers.Conv2D(1, 1, padding='same', activation = 'sigmoid')(ai)
ak.shape

TensorShape([1, 8, 256, 256, 1])

#### Model assumbly

In [40]:
inputs = layers.Input(shape=((16,256,256,1)))
b = tf.keras.layers.Conv3D(32, (3,7,7), padding = 'same', activation='relu', bias_initializer=Constant(0.01), 
                           )(inputs)
c = tf.keras.layers.Conv3D(32, (3,5,5), padding = 'same', activation='relu', bias_initializer=Constant(0.01) 
                           )(b)
d = layers.MaxPooling3D((2,4,4))(c)
d = layers.Dropout(0.3)(d)
e = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(d)
f = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(e)
g = layers.MaxPooling3D((2,2,2))(f)
g = layers.Dropout(0.3)(g)
h = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(g)
i = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(h)
j = layers.MaxPooling3D((2,2,2))(i)
j = layers.Dropout(0.3)(j)
k = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(j)
l = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(k)
m = layers.MaxPooling3D((1,4,4))(l)
m = layers.Dropout(0.3)(m)
n = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(m)
o = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(n)
p = layers.MaxPooling3D((1,2,2))(o)
p = layers.Dropout(0.3)(p)
q = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(p)
r = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(q)
s = layers.MaxPooling3D((2,2,2))(r)
s = layers.Dropout(0.3)(s)
t = layers.Conv3DTranspose(512, (2,2,2), (2,2,2), padding = 'same', activation='relu')(s)
t = layers.concatenate([t, p])
t = layers.Dropout(0.3)(t)
u = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(t)
v = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(u)
w = layers.Conv3DTranspose(512, (2,2,2), (1,2,2), padding = 'same', activation='relu')(v)
w = layers.concatenate([w, o])
w = layers.Dropout(0.3)(w)
x = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(w)
y = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(x)
z = layers.Conv3DTranspose(256, (2,2,2), (1,4,4), padding = 'same', activation='relu')(y)
z = layers.concatenate([z, l])
z = layers.Dropout(0.3)(z)
aa = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(z)
ab = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(aa)
ac = layers.Conv3DTranspose(128, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ab)
ac = layers.concatenate([ac, i])
ac = layers.Dropout(0.3)(ac)
ad = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(ac)
ae = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(ad)
af = layers.Conv3DTranspose(64, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ae)
af = layers.concatenate([af, f])
af = layers.Dropout(0.3)(af)
ag = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(af)
ah = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(ag)
ai = layers.Conv3DTranspose(32, (2,2,2), (1,4,4), padding = 'same', activation='relu')(ah)
ag = layers.MaxPooling3D((2,1,1))(c)
ai = layers.concatenate([ai, ag])
outputs = layers.Conv2D(1, 1, padding='same', activation = 'sigmoid')(ai)

In [41]:
forecast_model = tf.keras.Model(inputs, outputs, name="3D_UNet")

In [42]:
# Verify output shape.
forecast_model.predict(a).shape

(1, 8, 256, 256, 1)

In [43]:
# Display model details.
forecast_model.summary()

Model: "3D_UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 256, 25  0           []                               
                                6, 1)]                                                            
                                                                                                  
 conv3d_22 (Conv3D)             (None, 16, 256, 256  4736        ['input_1[0][0]']                
                                , 32)                                                             
                                                                                                  
 conv3d_23 (Conv3D)             (None, 16, 256, 256  76832       ['conv3d_22[0][0]']              
                                , 32)                                                       

                                                                                                  
 dropout_6 (Dropout)            (None, 2, 2, 2, 102  0           ['concatenate_6[0][0]']          
                                4)                                                                
                                                                                                  
 conv3d_34 (Conv3D)             (None, 2, 2, 2, 512  4194816     ['dropout_6[0][0]']              
                                )                                                                 
                                                                                                  
 conv3d_35 (Conv3D)             (None, 2, 2, 2, 512  2097664     ['conv3d_34[0][0]']              
                                )                                                                 
                                                                                                  
 conv3d_tr

Total params: 34,987,457
Trainable params: 34,987,457
Non-trainable params: 0
__________________________________________________________________________________________________


In [44]:
# Compile model.
forecast_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
# Data loaders.
meta = pd.read_csv('Sample_Dataset/meta.csv')
t_gen = data_generator(meta, batch_size = 4)
forecast_model.fit(t_gen, epochs = 100)

Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/3D_UNet/conv3d_23/Conv3D/Conv3DBackpropFilterV2' defined at (most recent call last):
    File "C:\Users\iamst\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\iamst\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\iamst\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\iamst\anaconda3\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
      app.start()
    File "C:\Users\iamst\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "C:\Users\iamst\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\iamst\anaconda3\lib\asyncio\base_events.py", line 570,